In [3]:
%pip install guidance
%pip install openai


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import guidance

guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

start to install package: redis
successfully installed package: redis
start to install package: redis_om
successfully installed package: redis_om


#### Guardrail the LLM output structure

In [140]:
program = guidance('''
once {{people}} said:
"Without guidance, AI will {{gen "story"}}
Counter argument: "{{gen "argument"}}
''')

quote = program(people="steve jobs")

quote


Stop program 
once steve jobs said:
"Without guidance, AI will do more harm than good." 
Counter argument: " AI can be used to improve the lives of people, from medical diagnosis to autonomous vehicles, and with the right guidance, it can be a powerful tool for good."

#### Restrict LLM output selection

In [144]:
options = ['Yes', 'No', 'Maybe']

program = guidance('''Is the following sentence offensive? 
Sentence: {{example}}
Answer:{{select "answer" options=options}}''')
                   
executed_program = program(example='your tacos taste can be improved', options=options)


Stop program Is the following sentence offensive? 
Sentence: your tacos taste can be improved 
Answer: No

#### If conditions & Hidden generation

In [151]:
options = ["yes", "no"]

program = guidance(
'''User: {{query}}

{{#block hidden=True~}}
Is the user query rude or offensive?
Answer: {{select "rude" options=options}}
{{~/block~}}

{{#if rude=="yes"}}Assistant: Please be polite{{~/if~}}
{{#if rude=="no"}}Assistant: {{gen "answer"}}{{~/if~}}
''')
executed_program = program(query='your tacos taste too salty', options=options)



Stop program User: your tacos taste too salty 

 Assistant: I'm sorry to hear that. Is there anything else I can do to help?

# Generate emails with if condition

In [9]:
priorities = ["low priority", "medium priority", "high priority"]

email = guidance('''    
{{#block hidden=True~}}
Here is the customer message we received: {{email}}
Please give it a priority score
priority score: {{select "priority" options=priorities}}
{{~/block~}}
        
{{#block hidden=True~}}
You are a world class customer support; Your goal is to generate an response based on the customer message;
Here is the customer message to respond: {{email}}
Generate an opening & one paragraph of response to the customer message based on priority score: {{priority}}:
{{gen 'email_response'}} 
{{~/block~}}

{{email_response}}
{{#if priority=='high priority'}}Would love to setup a call this/next week, here is the calendly link: https://calendly.com/jason-zhou{{/if}}
Best regards
Jason
''')

email_response = email(email='What features does webflow have but wix dont have?', priorities=priorities)


Stop program 
 
Thanks for your question! Webflow has a lot of features that Wix doesn't have. One of the most notable features is the ability to create custom code. This allows you to create more complex websites with custom functionality. Additionally, Webflow also offers a wide range of design tools, including a visual editor, custom fonts, and more. 
 
Best regards
Jason

# Generate chart

In [11]:
import urllib.parse

def generate_chart(query):
    
    def parse_chart_link(chart_details):
        encoded_chart_details = urllib.parse.quote(chart_details, safe='')

        output = "![](https://quickchart.io/chart?c=" + encoded_chart_details + ")"

        return output
    
    examples = [
        {
            'input': "Make a chart of the 5 tallest mountains",
            'output': {"type":"bar","data":{"labels":["Mount Everest","K2","Kangchenjunga","Lhotse","Makalu"], "datasets":[{"label":"Height (m)","data":[8848,8611,8586,8516,8485]}]}}
        },
        {
            'input': "Create a pie chart showing the population of the world by continent",
            'output': {"type":"pie","data":{"labels":["Africa","Asia","Europe","North America","South America","Oceania"], "datasets":[{"label":"Population (millions)","data": [1235.5,4436.6,738.8,571.4,422.5,41.3]}]}}
        }
    ]

    guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

    chart = guidance(
    '''    
    {{#block hidden=True~}}
    You are a world class data analyst, You will generate chart output based on a natural language;

    {{~#each examples}}
    Q:{{this.input}}
    A:{{this.output}}
    ---
    {{~/each}}
    Q:{{query}}
    A:{{gen 'chart' temperature=0 max_tokens=500}}    
    {{/block~}}
    
    Hello here is the chart you want
    {{parse_chart_link chart}}
    ''')

    return chart(query=query, examples=examples, parse_chart_link=parse_chart_link)


In [12]:
chart = generate_chart("Create a pie chart showing the population of the world by continent")

chart

Stop program 
 Hello here is the chart you want
 ![](https://quickchart.io/chart?c=%20%7B%27type%27%3A%20%27pie%27%2C%20%27data%27%3A%20%7B%27labels%27%3A%20%5B%27Africa%27%2C%20%27Asia%27%2C%20%27Europe%27%2C%20%27North%20America%27%2C%20%27South%20America%27%2C%20%27Oceania%27%5D%2C%20%27datasets%27%3A%20%5B%7B%27label%27%3A%20%27Population%20%28millions%29%27%2C%20%27data%27%3A%20%5B1235.5%2C%204436.6%2C%20738.8%2C%20571.4%2C%20422.5%2C%2041.3%5D%7D%5D%7D%7D)

# Embed text to image 

In [117]:
def generate_story(story_idea):
        
    story = guidance('''
{{#block hidden=True~}}
You are a world class story teller; Your goal is to generate a short tiny story less than 200 words based on a story idea;

Here is the story idea: {{story_idea}}
Story: {{gen 'story' temperature=0}}
{{/block~}}

{{#block hidden=True~}}
You are a world class AI artiest who are great at generating text to image prompts for the story; 
Your goal is to generate a good text to image prompt and put it in a url that can generate image from the prompt;

Story: You find yourself standing on the deck of a pirate ship in the middle of the ocean. You are checking if there are still people on the ship
Image url markdown: ![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20man%20standing%20on%20the%20deck%20of%20a%20pirate%20ship%20in%20the%20middle%20of%20the%20ocean)
                
Story: {{story}}
Image url markdown: {{gen 'image_url' temperature=0 max_tokens=500}})
{{~/block~}}
                     
Story: {{~story~}}
{{image_url}}
''')

    story = story(story_idea=story_idea)
    print(story)
    return story

In [118]:
generate_story("little girl dreaming")

Stop program 
Story: 

The little girl lay in her bed, her eyes closed and her breathing slow and steady. She was dreaming of a world far away, a world of adventure and excitement. In her dream, she was soaring through the sky, the wind in her hair and the sun on her face. She was free, and nothing could stop her.

She flew over mountains and valleys, forests and rivers, and she felt like she could go on forever. She was so high up that she could see the entire world below her, and it was beautiful.

The little girl smiled in her sleep, content in the knowledge that she could go anywhere she wanted in her dreams. She was the master of her own destiny, and she could do anything she wanted.

The little girl's dream eventually faded away, but the feeling of freedom and joy stayed with her. She knew that no matter what happened in the real world, she could always escape to her dream world and be free. 
![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20little%20girl%20lying%20in%20bed%20dreaming%20of%20a%20world%20far%20away%20with%20the%20wind%20in%20her%20hair%20and%20the%20sun%20on%20her%20face)